In [49]:
import nltk
import evaluate
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments
from datasets import load_dataset, concatenate_datasets, DatasetDict

# Dataset

In [2]:
dataset_checkpoint = "amazon_reviews_multi"
dataset_commit_id = "f256e74ee2353b7c7854f86f86200f220531caa4"

In [3]:
spanish_dataset = load_dataset(dataset_checkpoint, revision=dataset_commit_id, name="es")
english_dataset = load_dataset(dataset_checkpoint, revision=dataset_commit_id, name="en")

In [4]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [5]:
spanish_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

## Explore

In [6]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n>> Title: {example['review_title']}")
        print(f">> Review: '{example['review_body']}'")

In [7]:
show_samples(english_dataset)


>> Title: Worked in front position, not rear
>> Review: '3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.'

>> Title: meh
>> Review: 'Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue'

>> Title: Can't beat these for the money
>> Review: 'Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. Some h

In [8]:
show_samples(spanish_dataset)


>> Title: .
>> Review: 'La montarlo se rompió una rueda debido a materiales débiles, pero al arreglarla funciona correctamente.'

>> Title: Primeras impresiones
>> Review: 'El servicio ha sido muy bueno, me ha llegado 2 días antes de lo previsto. En cuanto al producto no es que me haya dado muy buenas primeras impresiones. El borde del protector es de plástico y lo único que hay de cristal es la pantalla. Además el plástico es muy fino. A nivel estético queda muy bien y se ajusta perfectamente, la única queja que tengo es eso, que no sea todo de cristal y que para mi gusto es demasiado fino. De la resistencia no tengo ni idea ya que es el primer día que lo llevo. No creo que sea mal producto del todo si no que depende del gusto y el cuidado que tenga cada uno de su móvil. Personalmente creo que por el mismo precio hay otros productos que si que son enteros de cristal y más gordos que por lo menos a mí me generan más confianza.'

>> Title: .
>> Review: 'Funciona genial y la llevo conmi

In [9]:
english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]

In [10]:
english_df["product_category"].value_counts()

product_category
home                        17679
apparel                     15951
wireless                    15717
other                       13418
beauty                      12091
drugstore                   11730
kitchen                     10382
toy                          8745
sports                       8277
automotive                   7506
lawn_and_garden              7327
home_improvement             7136
pet_products                 7082
digital_ebook_purchase       6749
pc                           6401
electronics                  6186
office_product               5521
shoes                        5197
grocery                      4730
book                         3756
baby_product                 3150
furniture                    2984
jewelry                      2747
camera                       2139
industrial_supplies          1994
digital_video_download       1364
luggage                      1328
musical_instruments          1102
video_games                   7

## Prepare

In [11]:
english_dataset.reset_format()

In [12]:
def filter_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )

In [13]:
english_books = english_dataset.filter(filter_books)
spanish_books = spanish_dataset.filter(filter_books)

In [14]:
show_samples(english_books)


>> Title: I'm dissapointed.
>> Review: 'I guess I had higher expectations for this book from the reviews. I really thought I'd at least like it. The plot idea was great. I loved Ash but, it just didnt go anywhere. Most of the book was about their radio show and talking to callers. I wanted the author to dig deeper so we could really get to know the characters. All we know about Grace is that she is attractive looking, Latino and is kind of a brat. I'm dissapointed.'

>> Title: Good art, good price, poor design
>> Review: 'I had gotten the DC Vintage calendar the past two years, but it was on backorder forever this year and I saw they had shrunk the dimensions for no good reason. This one has good art choices but the design has the fold going through the picture, so it's less aesthetically pleasing, especially if you want to keep a picture to hang. For the price, a good calendar'

>> Title: Helpful
>> Review: 'Nearly all the tips useful and. I consider myself an intermediate to advance

In [15]:
books_dataset = DatasetDict()

for split in english_books.keys():
    books_dataset[split] = concatenate_datasets(
        [english_books[split], spanish_books[split]]
    )
    books_dataset[split] = books_dataset[split].shuffle(seed=42)

In [16]:
show_samples(books_dataset)


>> Title: Easy to follow!!!!
>> Review: 'I loved The dash diet weight loss Solution. Never hungry. I would recommend this diet. Also the menus are well rounded. Try it. Has lots of the information need thanks.'

>> Title: PARCIALMENTE DAÑADO
>> Review: 'Me llegó el día que tocaba, junto a otros libros que pedí, pero la caja llegó en mal estado lo cual dañó las esquinas de los libros porque venían sin protección (forro).'

>> Title: no lo he podido descargar
>> Review: 'igual que el anterior'


## TODO: make word count distribution plots

## Filter low-wordcount review titles

In [17]:
# Use white space heuristic for "word count"
books_dataset = books_dataset.filter(lambda x: len(x["review_title"].split()) > 2)

## Tokenization

In [18]:
model_checkpoint = "google/mt5-small"
model_commit_id = "38f23af8ec210eb6c376d40e9c56bd25a80f195d"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, revision_id=model_commit_id)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/home/carcook/dev/transformers-course/myenv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [20]:
inputs = tokenizer("I think Catch 22 is the best book ever.")

In [21]:
inputs

{'input_ids': [336, 5231, 259, 139068, 1024, 339, 287, 1920, 3435, 14049, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I',
 '▁think',
 '▁',
 'Catch',
 '▁22',
 '▁is',
 '▁the',
 '▁best',
 '▁book',
 '▁ever',
 '.',
 '</s>']

In [23]:
max_input_length = 512
max_target_length = 30

In [24]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True
    )
    labels = tokenizer(
        examples["review_title"],
        max_length=max_target_length,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [25]:
tokenized_datasets = books_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

# Evaluation metrics

In [26]:
generated_summary = "I absolutely loved reading Catch 22"
reference_summary = "I loved reading Catch 22"

In [30]:
rouge_score = evaluate.load("rouge")

In [33]:
scores = rouge_score.compute(
    predictions=[generated_summary],
    references=[reference_summary],
)

In [34]:
scores

{'rouge1': [0.9090909090909091],
 'rouge2': [0.6666666666666665],
 'rougeL': [0.9090909090909091],
 'rougeLsum': [0.9090909090909091]}

## Baseline

In [37]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/carcook/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]
    return metric.compute(predictions=summaries, references=dataset["review_title"])

In [42]:
three_sentence_summary(books_dataset["train"][1]["review_body"])

'I grew up reading Koontz, and years ago, I stopped,convinced i had "outgrown" him.\nStill,when a friend was looking for something suspenseful too read, I suggested Koontz.\nShe found Strangers.'

In [44]:
score = evaluate_baseline(books_dataset["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict(
    (rn, round(score[rn] * 100, 2)) for rn in rouge_names
)

In [45]:
rouge_dict

{'rouge1': 16.8, 'rouge2': 8.93, 'rougeL': 15.61, 'rougeLsum': 15.99}

# Fine-tuning

In [48]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, revision=model_commit_id)

In [52]:
batch_size = 8
num_train_epochs = 8
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]
output_dir = f"../temp/07/{model_name}-finetuned-amazon-en-es"

In [53]:
args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=False
)